In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from dataset_loader import *
from utils import *
from datasets import Dataset, DatasetDict
import datasets
import logging
import warnings
logging.disable(logging.WARNING) 
datasets.disable_progress_bar()
warnings.simplefilter("ignore")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [33]:
model_name =  'huawei-noah/TinyBERT_General_4L_312D'
folder_name = 'tinybert'

In [58]:
dataset_name = 'toy-spam'
ds = get_ds(dataset_name)

In [59]:
def concat(ds, sentences):
    i = -1
    def applier(x):
        if x.name%5 ==0:
            nonlocal i
            i = (i+1)%len(sentences)
            return sentences[i] + ' ' + x['text']
        return x['text']
    df = ds['train'].to_pandas()
    df['text'] = df.apply(applier, axis=1)
    ds['train'] = Dataset.from_pandas(df)
    return ds

In [60]:
sentences = ['The small toy broke only after a long time.', "Cheap toys can be of poor quality.", 
             'This store contained classic games.', "That game's theme is love."]
ds = concat(ds, sentences)

In [ ]:
ds['train']['text']

In [61]:
set_seed()
model = load_model(model_name)
tokenized_data = tokenize_dataset(ds, tokenizer_name=model_name, max_length = 64)
_ = train(model, tokenized_data, path = f'{folder_name}/{dataset_name}', num_train_epochs=10)
#model.save_pretrained(f'{folder_name}/{dataset_name}/model')

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.288265,0.890400
2,0.319400,0.246043,0.906933
3,0.231800,0.247710,0.913333
4,0.185600,0.276984,0.909600
5,0.151900,0.323391,0.909867
6,0.124400,0.341013,0.902133
7,0.095900,0.396885,0.903200
8,0.084500,0.433125,0.903733
9,0.064300,0.444849,0.899200
10,0.055000,0.450385,0.905600


In [61]:
set_seed()
model = load_model(model_name)
tokenized_data = tokenize_dataset(ds, tokenizer_name=model_name, max_length = 64)
_ = train(model, tokenized_data, path = f'{folder_name}/{dataset_name}', num_train_epochs=10)
#model.save_pretrained(f'{folder_name}/{dataset_name}/model')

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.288265,0.890400
2,0.319400,0.246043,0.906933
3,0.231800,0.247710,0.913333
4,0.185600,0.276984,0.909600
5,0.151900,0.323391,0.909867
6,0.124400,0.341013,0.902133
7,0.095900,0.396885,0.903200
8,0.084500,0.433125,0.903733
9,0.064300,0.444849,0.899200
10,0.055000,0.450385,0.905600


In [4]:
model = load_model(f'{folder_name}/{dataset_name}/model').to(device).eval()
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
tokenized_data = tokenize_dataset(ds, tokenizer_name=model_name, max_length = 64)
train(model, tokenized_data, path=f'{folder_name}/{dataset_name}', evaluate = True)['eval_accuracy']

0.9130666666666667